In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py
# exists and contains get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
from random import random
from itertools import count
from argparse import ArgumentParser
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

The game of life offers a simplistic model of living cells coming to life, surviving, and dying. It is played on a 2-dimensional grid. We first assume that the grid is bounded. To start with, it supports an initial population distribution. Every living cell has at most 8 neighbouring living cells (positioned horizontally, vertically, and diagonally). The population evolves in stages. At every stage:
* living cells surrounded by exactly 2 or 3 living cells survive; with less than 2, they die of loneliness, and with more than 3, they die of overcrowding;
* empty positions surrounded by exactly 3 neighbouring living cells are nice and cosy, and see a cell come to life.

For instance, consider a grid of size at least 5 x 5 whose center is initially populated as follows:

$\begin{array}{cccccc}
      &       &       &       &       \\
      & \star & \star & \star &       \\
      & \star &       & \star &       \\
      & \star & \star & \star &       \\
      &       &       &       &
\end{array}$

For the next three stages, the population distribution will become first

$\begin{array}{cccccc}
      &       & \star &       &       \\
      & \star &       & \star &       \\
\star &       &       &       & \star \\
      & \star &       & \star &       \\
      &       & \star &       &
\end{array}$

and then

$\begin{array}{cccccc}
      &       & \star &       &       \\
      & \star & \star & \star &       \\
\star & \star &       & \star & \star \\
      & \star & \star & \star &       \\
      &       & \star &       &
\end{array}$

and then

$\begin{array}{cccccc}
      & \star & \star & \star &       \\
\star &       &       &       & \star \\
\star &       &       &       & \star \\
\star &       &       &       & \star \\
      & \star & \star & \star &       \\
\end{array}$

Let us work with a grid of size 10 x 10. First, let us create a grid for a lifeless world, filled with nothing but 0's. Let us also write a function to display the contents of the grid:

In [ ]:
size = 10
grid = [[0] * size for _ in range(size)]

def display_grid():
    for row in grid:
        print(*row)
        
display_grid()

We want the grid to store some 1's that represent living cells. We fix an expected density of living cells, say 30%, and 100 times, generate a number between 0 and 1, for a living cell if the generated number is less than 0.3, and for an empty position otherwise. Let us first initialise __grid__ to a list of 10 lists of 10 __0__'s, before each member of __grid__'s 10 members is given a chance of 0.3 to be changed to a __1__:

In [ ]:
density = 0.3
grid = [[0 for _ in range(size)] for _ in range(size)]
for i in range(size):
    for j in range(size):
        if random() < density:
            grid[i][j] = 1

display_grid()

This could have been more concisely achieved as follows:

In [ ]:
grid = [[None] * size for _ in range(size)]
for i in range(size):
    for j in range(size):
        grid[i][j] = int(random() < density)

display_grid()

But the following code fragment would not be appropriate:

In [ ]:
grid = [[None] * size] * size
for i in range(size):
    for j in range(size):
        grid[i][j] = int(random() < density)

display_grid()

Indeed,
* __0__ denotes the representation in memory of a structure $Z$ that models the integer 0;
* __[0] \* size__ denotes the representation in memory of a list $L$ consisting of 10 times that representation in memory of $Z$;
* __[[0] \* size] \* size__ denotes the representation in memory of a list $L'$ consisting of 10 times that representation in memory of $L$.

A statement of the form __grid[i][j]__ then lets one of the 10 members of $L$ change from a representation in memory of $Z$ to a representation in memory of a structure $U$ that models the integer 1. This change in $L$ is reflected in the 10 denotations of $L$ in $L'$. The following code fragment illustrates further:

In [ ]:
L = [[0] * 3] * 2; L
L[0][1] = 10; L
L[1][2] = 20; L

That being said, using two list comprehensions is the best way to define a list of lists of randomly generated __0__'s and __1__'s following the desired distribution:

In [ ]:
density = 0.3
grid = [[int(random() < density) for _ in range(size)]
                                          for _ in range(size)
       ]

display_grid()

Let us write an alternative to __display_grid()__ for a nicer output, black and white squares rather than 1's and 0's:

In [ ]:
def display_population():
    squares = {0: '\u2b1c', 1: '\u2b1b'}
    for row in grid:
        print(''.join(f'{squares[e]}' for e in row))

display_population()

To compute the next stage of living cells distribution, it suffices to define a new grid meant to represent, at the intersection of row $i$ and column $j$, a living cell if there is currently a living cell at that location and there are exactly 2 neighbouring living cells, or if there are exactly 3 neighbouring living cells (either there is a living cell at that location and it survives, or there is no living cell at that location and one appears). Eventually, we make the original grid denote the new grid (necessitating the __global grid__ statement). When computing neighbours, we have to pay attention to positions that have only 5 or 3 neighbours rather than 8 because they are along one of the grid's boundaries or at one of grid's corners:

In [ ]:
def next_generation():
    global grid
    new_grid = [[None] * size for _ in range(size)]
    for i in range(size):
        for j in range(size):
            nb_of_neighbours = 0
            # Above left
            if i and j and grid[i - 1][j - 1]:
                nb_of_neighbours += 1
            # Above
            if i and grid[i - 1][j]:
                nb_of_neighbours += 1
            # Above right
            if i and j < size - 1 and grid[i - 1][j + 1]:
                nb_of_neighbours += 1
            # Left
            if j and grid[i][j - 1]:
                nb_of_neighbours += 1
            # Right
            if j < size - 1 and grid[i][j + 1]:
                nb_of_neighbours += 1
            # Below left
            if i < size - 1 and j and grid[i + 1][j - 1]:
                nb_of_neighbours += 1
            # Below
            if i < size - 1 and grid[i + 1][j]:
                nb_of_neighbours += 1
            # Below right
            if i < size - 1 and j < size - 1 and grid[i + 1][j + 1]:
                nb_of_neighbours += 1
            new_grid[i][j] = int(grid[i][j] and nb_of_neighbours == 2 or
                                 nb_of_neighbours == 3
                                )
    grid = new_grid
    
next_generation()
display_population()
print()

next_generation()
display_population()

The code is awkward because of the care exercised for the boundary positions. We can instead work with a grid of size 12 x 12, having in mind a surrounding "frame" (first row, last row, first column, last column), where no cell will ever come to life, but that provides exactly 8 neighbours to all positions within the frame. The cost of using a bit of extra space is more than compensated by the gain in elegance, and to a lesser extent, efficiency:

In [ ]:
framed_grid = [[0] * (size + 2) for _ in range(size + 2)]
for i in range(1, size + 1):
    for j in range(1, size + 1):
        framed_grid[i][j] = int(random() < density)

def display_population_within_frame():
    squares = {0: '\u2b1c', 1: '\u2b1b'}
    for i in range(1, size + 1):
        print(''.join(f'{squares[framed_grid[i][j]]}'
                                      for j in range(1, size + 1)
                     )
             )
        
def next_generation_within_frame():
    global framed_grid
    new_framed_grid = [[0] * (size + 2) for _ in range(size + 2)]
    for i in range(1, size + 1):
        for j in range(1, size + 1):
            nb_of_neighbours = sum((framed_grid[i - 1][j - 1],
                                    framed_grid[i - 1][j],
                                    framed_grid[i - 1][j + 1],
                                    framed_grid[i][j - 1],
                                    framed_grid[i][j + 1],
                                    framed_grid[i + 1][j - 1],
                                    framed_grid[i + 1][j],
                                    framed_grid[i + 1][j + 1]
                                   )
                                  )
            new_framed_grid[i][j] = int(framed_grid[i][j] and
                                        nb_of_neighbours == 2 or
                                        nb_of_neighbours == 3
                                       )
    framed_grid = new_framed_grid

display_population_within_frame()
print()

next_generation_within_frame()
display_population_within_frame()
print()

next_generation_within_frame()
display_population_within_frame()

A Python list can contain all kinds of values, of all types. This flexibility comes at the cost of efficiency. When the members of a list are all of the same type, then the __numpy__ module and its __array__ class offer more powerful operations, that are implemented very effectively. The shape of an __array__ can change to treat a sequence of data all of the same type, stored consecutively in memory, as a list, or as a list of lists, or as a list of lists of lists... The following illustrates:

In [ ]:
# 1 dimension
L = np.array([0, 1, 2, 3, 4, 5, 6, 7])

# Type inferred from the values used for initialisation,
# size is number of elements, independy of shape,
# which represents the number of dimensions, and
# the number of elements in each dimension
L, L.dtype, L.size, L.shape
# Viewed in 2 dimensions, in various ways
L.reshape((1, 8))
L.reshape((1, 8))
L.reshape((8, 1))
L.reshape((2, 4))
L.reshape((4, 2))
# Viewed in 3 dimensions
L.reshape((2, 2, 2))
L
# Changing the element of index 0 of each of the three dimensions
L.reshape(2, 2, 2)[0, 0, 0] = 10
# L has been changed, it was all the same data viewed differently
L

Consistently with the fact that the shape of an __array__ is only a matter of viewing in a particular way a linear sequence of data, and can change dynamically, elements or sections of an __array__ will be denoted using a single pair of square brackets with in between, as many indexes or slices are there are dimensions in the current view. This syntax has been used in the last assignment of the previous code fragment. Observe that the number of slices between square brackets corresponds to the number of dimensions of the view of the extracted data:

In [ ]:
L = np.array(range(16))

L
L[10]
L[8: 13]

In [ ]:
L = np.array(range(16)).reshape(2, 8)

L
L[1, 2]
L[1, :]
L[1: 2, :]
L[: , 3]
L[: , 3: 4]

In [ ]:
L = np.array(range(16)).reshape(2, 2, 4)

L
L[1, 0, 2]
L[1, 1, 1: ]
L[1, : , 1: 3]
L[0: 1, 0: 1, 1: 3]

The __zeroes__ function of the __numpy__ module provides a convenient way to define an __array__ of a given shape, with elements all initialised to 0:

In [ ]:
np.zeros((2, 2))
np.zeros((2, 2), np.int)

Assigning a given value to all members of a section of an __array__ is more easily done than with Python lists. Note that __np_grid__ below is defined so as to represent the "framed" version of the grid depicted at the very beginning:

In [ ]:
np_framed_grid = np.zeros((7, 7), np.int)
np_framed_grid[2: 5, 2: 5] = 1
np_framed_grid[3, 3] = 0

# A "framed" grid
np_framed_grid
# The "inside" of the grid, without the surrounding "frame"
np_framed_grid[1: -1, 1: -1]

Notice the following:

* If a copy of the grid is shifted one position to the right and one position below, then every position "inside the frame" of the grid will be aligned with a 1 or a 0 of the shifted copy depending on whether there is a neighbouring living cell to the left of and above that position.
* If a copy of the grid is shifted one position to the right, then every position of the grid "inside the frame" will be aligned with a 1 or a 0 of the shifted copy depending on whether there is a neighbouring living cell to the left of that position.
* If a copy of the grid is shifted one position to the right and one position above, then every position "inside the frame" of the grid will be aligned with a 1 or a 0 of the shifted copy depending on whether there is a neighbouring living cell to the left of and below that position.

The following code fragment defines three __arrays__'s that represent that part of the three shifted grids that is positioned "inside the frame" of the grid:

In [ ]:
np_above_left = np_framed_grid[: -2, : -2]
np_left = np_framed_grid[1: -1, : -2]
np_below_left = np_framed_grid[2: , : -2]

np_above_left
np_left
np_below_left

Similarly, for the copies of the grids shifted one position below or above, to align the positions of the grid "inside the frame" with their neighbouring living cells, if any, above or below, respectively, we get:

In [ ]:
np_above = np_framed_grid[: -2, 1: -1]
np_below = np_framed_grid[2: , 1: -1]

np_above
np_below

Similarly, for the copies of the grids shifted one position to the left and one position below, or one position to the left, or one position to the left and one position above, to align the positions of the grid "inside the frame" with their neighbouring living cells, if any, to the right and above, to the right, or to the right and below, respectively, we get:

In [ ]:
np_above_right = np_framed_grid[: -2, 2:]
np_right = np_framed_grid[1: -1, 2:]
np_below_right = np_framed_grid[2: , 2:]

np_above_right
np_right
np_below_right

Adding __array__'s of the same shape corresponds to matrix addition. In other words, adding up the 8 __array__'s just defined yields an __array__ that represents the number of neighbouring living cells at any position "inside the frame":

In [ ]:
np_above_left + np_left + np_below_left +\
np_above + np_below +\
np_above_right + np_right + np_below_right

Boolean operators can also be applied element for element to two __array__'s of the same shape:

In [ ]:
A = np.array((0, 0, 1, 2)).reshape(2, 2)
B = np.array((True, False, True, False)).reshape(2, 2)

A
B
np.logical_or(A, B)
np.logical_and(A, B).astype(np.int)

So the key elements of the game of life can be alternatively implemented using __array__'s as follows:

In [ ]:
framed_grid = np.zeros((size + 2, size + 2), np.int)
for i in range(1, size - 1):
    for j in range(1, size - 1):
        framed_grid[i, j] = int(random() < density)

def next_generation_within_frame():
    global framed_grid
    number_of_neighbours = np.zeros((size + 2, size + 2))
    number_of_neighbours[1: -1, 1: -1] =\
                framed_grid[: -2, : -2] + framed_grid[: -2, 1: -1] +\
                framed_grid[: -2, 2:] + framed_grid[1: -1, : -2] +\
                framed_grid[1: -1, 2:] + framed_grid[2: , : -2] +\
                framed_grid[2: , 1: -1] + framed_grid[2: , 2:]
    framed_grid = np.logical_or(np.logical_and(framed_grid == 1,
                                               number_of_neighbours == 2
                                              ),
                                number_of_neighbours == 3
                               ).astype(np.int)

display_population_within_frame()
print()

next_generation_within_frame()
display_population_within_frame()
print()

next_generation_within_frame()
display_population_within_frame()

Let us see how to create movies of population evolutions, starting from initial populations read from rle files (rle is for "run length encoded"); __gosperglidergun.rle__ is an example of such a file, that the __cat__ Jupyter magic command lets us display:

In [ ]:
cat gosperglidergun.rle

The first line after the comment lines indicates that a pattern that occupies 36 columns and 9 rows will follow. (B3/S23 is for Birth if 3 neighbouring living cells, Survival if 2 or 3 neighbouring living cells, so a description of the rules of the game of life.) Then comes the description of the pattern. Such a description can span many lines. It ends in an exclamation mark. It consists of:

* __\$__ to indicate moving one row below, or in case a positive integer $k$ precedes __\$__, $k$ rows below;
* __b__ to indicate an empty position, or in case a positive integer $k$ precedes __b__, $k$ consecutive empty positions;
* __o__ to indicate a living cell, or in case a positive integer $k$ precedes __o__, $k$ consecutive living cells.

So the pattern described in __gosperglidergun.rle__ has a first row consisting of 24 empty positions followed by 1 living cell followed by 11 empty positions, for a total of 36 positions, but it is not necessary to indicate trailing empty positions, so rather than __24bo11b\$__, we could have read __24bo\$__. We see this abbreviation used for the last row, where 12 empty positions are followed by 2 living cells; the 12 empty positions at the end are implicit.

The first pieces of information to extract from __gosperglidergun.rle__ seem to be the values of __x__ and __y__. They are not indispensable, but due to the potential use of the abbreviation previously mentioned, the pattern description can be more easily processed if __x__'s value is known, and knowing __y__'s value too does not harm. We take advantage of the __re__ module and its __search()__ function:

In [ ]:
with open('gosperglidergun.rle') as file:
    for line in file:
        if line.startswith('#'):
            continue
        x, y = (int(e) for e in re.search('(\d+)[^\d]*(\d+)',
                                          line
                                         ).groups())
        break

x, y

The following lines can all be read and stripped of any space at both ends, including the final new line character, then concatenated into a new single string, from which a string without the final exclamation mark can be obtained and split into substrings separated by a dollar sign:

In [ ]:
with open('gosperglidergun.rle') as file:
    for line in file:
        if line.startswith('#'):
            continue
        x, y = (int(e) for e in re.search('(\d+)[^\d]*(\d+)',
                                          line
                                         ).groups())
        break
    lines = ''.join(line.strip() for line in file)[: -1].split('$')
lines

The previous implementation of the evolution of a population assumed that the game of life is played on a finite grid whose boundaries have their role in how life evolves; for instance, a cluster of living cell could bounce back on a grid boundary. As the comment in __gosperglidergun.rle__ indicates, part of the interest in the game of life has been to find patterns "with unbounded growth". To be faithful to the interpretation of the game where the grid is not the whole universe, but only a finite window in an unbounded universe, we will create movies supposed to run for a predetermined number of generations, with a default value of 1,000 that the user will be able to change, but the movie will stop as soon as a living cell touches on the "frame" of the grid, if that happens before the movie is set to end:

In [ ]:
max_nb_of_iterations = 1_000

The movie is all the more likely to end before __max_nb_of_iterations__ many evolution steps have taken place that there is little free space around the area occupied by the pattern decoded from the rle file. By default, we set the horizontal and vertical dimensions of the grid to 5 times the max of __x__ and __y__, but the user will also be able to change this default value:

In [ ]:
pattern_size_multiplier = 5
size = max(x, y) * pattern_size_multiplier

size

To position the rectangular part of the grid occupied by the pattern roughly at the centre, we set the coordinates __i__ (row index) and __j__ (column index) of the top left corner of that rectangular area accordingly:

In [ ]:
i, j = (size - y) // 2, (size - x) // 2

i, j

Each member of __lines__ needs to be analysed to change some __0__'s to __1__'s in __grid__. For better illustration, we set __lines__, __i__ and __j__ manually to the following values:

lines = ['', '50', 'b', '2b50', 'o50', '2o', 'b2o5bob8o10b', '2o5b8obo11b13o50']  
i = 0  
j = 0

* The first member of lines, namely, __''__, should make __i__ become 1 (0+1). There is no __0__ to change to __1__ on the first row of the grid.
* The second member of lines, namely, __'50'__, should make __i__ become 51 (1+50). There is no __0__ to change to __1__ on the next 50 rows of the grid.
* The third member of lines, namely, __'b'__, should make __i__ become 52 (51+1). There is no __0__ to change to __1__ on the 52nd row of the grid.
* The fourth member of lines, namely, __'2b50'__, should make __i__ become 102 (52+50). There is no __0__ to change to __1__ on the 53rd row of the grid, nor on the next 50 rows of the grid.
* The fifth member of lines, namely, __'o50'__, should make __i__ become 152 (102+50). On the 103rd row of the grid, the __0__'s between positions 0 included and 1 (0+1) excluded should be changed to __1__.
* The sixth member of lines, namely, __'2o'__, should make __i__ become 153 (152+1). On the 153rd row of the grid, the __0__'s between positions 0 included and 2 (0+2) excluded should be changed to __1__.
* The seventh member of lines, namely, __'b2o5bob8o10b'__, should make __i__ become 154 (153+1). On the 154th row of the grid, the __0__'s between positions 1 included and 3 (1+2) excluded, between positions 8 (3+5) included and 9 (8+1) excluded, and between positions 10 (9+1) included and 18 (10+8) excluded, should be changed to __1__.
* The eighth member of lines, namely, __'2o5b8obo11b13o50'__, should make __i__ become 204 (154+50). On the 155th row of the grid, the __0__'s between positions 0 included and 2 (0+2) excluded, between positions 7 (2+5) included and 15 (7+8) excluded, between positions 16 (15+1) included and 17 (16+1) excluded, and between positions 28 (17+11) included and 41 (28+13) excluded, should be changed to __1__.

To achieve this, we can, for each member of __lines__, separate the digits at the end, if any, from the rest:
* ('', '')
* ('', '50')
* ('b', '')
* ('2b', '50')
* ('o', '50')
* ('2o', '')
* ('b2o5bob8o10b', '')
* ('2o5b8obo11b13o', '50')

The second member of each pair indicates which value __i__ should be increased by (1 if empty string). The first member of each pair except the first two, which are empty, can then be split using __b__ as a separator:
* ('', '')
* ('2', '')
* ('o',)
* ('2o',)
* ('', '2o5', 'o', '8o10', '')
* ('2o5', '8o', 'o11', '13o')

For those of the previous tuples that end in an empty string, that empty string indicates that there is no further living cell on the corresponding row, and there is no need to process that string, so it is enough to process:
* ('',)
* ('2',)
* ('o',)
* ('2o',)
* ('', '2o5', 'o', '8o10')
* ('2o5', '8o', 'o11', '13o')

In each of the previous tuples, each nonempty string $s$ where __o__ occurs indicates first a number of living cells (1 if $s$ is empty or starts with __o__), and in case $s$ is not the tuple's last member, a number of empty positions to follow (1 if $s$ ends in __o__). The three strings where __o__ does not occur all start a tuple, and indicate the position (1 if $s$ is empty) occupied that the leftmost living cell on that row, if any; all other rows definitely have at least one living cell, the leftmost of which occupies position 0. So from the previous tuples, one can derive the following sequences of start positions of runs of living cells or runs of empty positions, beginning with a run of living cells, if any:
* [1]
* [2]
* [0, 1, 2]
* [0, 2, 3]
* [1, 3, 8, 9, 10, 18, 28]
* [0, 2, 7, 15, 16, 17, 28, 41, 42]

The runs of living cells are then described by the following pairs, who first element indicates the start of the run and whose second element indicates the position past the end of the run:
* []
* []
* [[0, 1]]
* [[0, 2]]
* [[1, 3], [8, 9], [10, 18]]
* [[0, 2], [7, 15], [16, 17], [28, 41]]

The next piece of code follows the approach just described up to the penultimate enumeration, and traces computation: 

In [ ]:
lines = '', '50', 'b', '2b50', 'o50', '2o',\
        'b2o5bob8o10b', '2o5b8obo11b13o50'
i, j = 0, 0

for line in lines:
    print()
    if not line:
        print('Case 0: line is empty')
        i += 1
        print('Make i equal to', i)
        continue
    if line.isdigit():
        print('Case 1: line is', line)
        i += int(line)
        print('Make i equal to ', i)
        continue
    print('Case 2: line is', line)
    line, nb_of_new_lines = re.match('(.*[^\d])(\d*)', line).groups()
    print('line becomes', line)
    try:
        i += int(nb_of_new_lines)
        print('\tCase 2.1: make i equal to', i)
    except ValueError:
        i += 1
        print('\tCase 2.2: make i equal to', i)
    line = line.split('b')
    print('\tAfter splitting with "b", line becomes:', line)
    if line[0].find('o') >= 0:
        run_spans = [j]           
        print('\t\tCase I: run_spans initialised to:', run_spans)
    else:
        try:
            run_spans = [j + int(line.pop(0))]
            print('\t\tCase II.1: run_spans initialised to:', run_spans)
            print('\t\tline becomes', line)
        except ValueError:
            run_spans = [j + 1]                
            print('\t\tCase II.2: run_spans initialised to:', run_spans)
            print('\t\tline becomes', line)
    for e in line:
        if not e:
            print('\t\tEmpty string, not processed')
            break
        print('\t\tProcessing', e, 'split with "o" to', e.split('o'))
        for run_span in e.split('o'):
            try:
                run_spans.append(run_spans[-1] + int(run_span))
                print('\t\t\tCase α, run_spans becomes:', run_spans)
            except ValueError:
                run_spans.append(run_spans[-1] + 1)
                print('\t\t\tCase β, run_spans becomes:', run_spans)

We can now put together all the code that reads the contents of an __.rle__ file, computes __x__ and __y__, defines __grid__ with the appropriate size, initialises it with __0__'s, and replaces some __0__'s by __1__'s as directed by the pattern in the __.rle__ file. We use the __gosperglidergun.rle__ file again:

In [ ]:
with open('gosperglidergun.rle') as file:
    for line in file:
        if line.startswith('#'):
            continue
        x, y = (int(e) for e in re.search('(\d+)[^\d]*(\d+)',
                                          line
                                         ).groups())
        break
    lines = ''.join(line.strip() for line in file)[: -1].split('$')
size = max(x, y) * pattern_size_multiplier
grid = np.zeros((size, size), np.int)
i, j = (size - y) // 2, (size - x) // 2
for line in lines:
    if not line:
        i += 1
        continue
    if line.isdigit():
        i += int(line)
        continue
    line, nb_of_new_lines = re.match('(.*[^\d])(\d*)', line).groups()
    line = line.split('b')
    if line[0].find('o') >= 0:
        run_lengths = [j]           
    else:
        try:
            run_lengths = [j + int(line.pop(0))]
        except ValueError:
            run_lengths = [j + 1]                
    for e in line:
        if not e:
            break
        for run_length in e.split('o'):
            try:
                run_lengths.append(run_lengths[-1] + int(run_length))
            except ValueError:
                run_lengths.append(run_lengths[-1] + 1)
    for n in range(len(run_lengths) // 2):
        grid[i, run_lengths[2 * n]: run_lengths[2 * n + 1]] = 1
    try:
        i += int(nb_of_new_lines)
    except ValueError:
        i += 1
print(grid[i - y: i, j: j + x])

To create the movie, we use __pyplot__ and __animation__ modules of __matplotlib__. The movie will consist of a __matplotlib.pyplot__ figure, displayed and then modified and displayed again at most __max_nb_of_iterations - 1__ many times. We set the time between the display of two successive frames to a default value of 20 milliseconds, that the user will be able to modify:

In [ ]:
animation_intervals = 20

We set the size of the figure to a default value of 10 by 10 inches, that the user will be able to modify as well. We define an appropriate resolution for our computer screen (in dpi's, or dots per inches). We do not want the x- and y- axes together with their ticks and values to be displayed. We want the figure to display __grid__, as well as the current stage of the evolution, in the form of red text positioned above the top left corner of the grid; the text is initialised to the empty string, but will become 0 when the first frame is created, and will increase by 1 for every new frame:

In [ ]:
figure_size = (10, 10)
figure = plt.figure(figsize = figure_size, dpi = 192)
plt.gca().get_xaxis().set_visible(False)
plt.gca().get_yaxis().set_visible(False)
population_state = plt.imshow(grid)
current_iteration = plt.text(5, -5, '', color = 'red')

We embed the code we wrote to compute the next state of the population distribution in a function __evolve()__ whose argument is expected to be a nonnegative integer that represents the stage of the evolution, starting with 0, and increasing by 1 at every stage. The methods __set_data()__ and __set_text()__ are called on __population_state__ and __current_iteration__, respectively, so as to modify them:

In [ ]:
def evolve(i):
    global grid
    number_of_neighbours = np.zeros(grid.shape)
    number_of_neighbours[1: -1, 1: -1] =\
            grid[: -2, : -2] + grid[: -2, 1: -1] + grid[: -2, 2:] +\
            grid[1: -1, : -2] + grid[1: -1, 2:]  +\
            grid[2: , : -2]   + grid[2: , 1: -1]  + grid[2: , 2:]
    grid = np.logical_or(np.logical_and(grid == 1,
                                        number_of_neighbours == 2
                                       ),
                         number_of_neighbours == 3
                        ).astype(np.int)
    population_state.set_data(grid)
    current_iteration.set_text(f'Iteration: {str(i)}')
    return population_state, current_iteration

We then define a generator function, __keep_going_or_stop()__, to yield forever all nonnegative integers, 0, 1, 2... but every time checking that no living cell is touching one of __grid__'s boundaries; would that be the case, the generator function returns __None__. Rather than defining a variable, initialising it to 0, and incrementing it by 1, we opt for a more elegant implementation and make use of the __count()__ iterable from the __itertools__ module. We first illustrate the use of __count()__:

In [ ]:
def test_count(start = None):
    if start is None:
        for i in count():
            if i > 3:
                return
            yield i
    else:
        for i in count(start):
            if i > start + 3:
                return
            yield i

        
list(test_count())
list(test_count(10))

We implement __keep_going_or_stop()__ as described:

In [ ]:
def keep_going_or_stop():
    for i in count():
        if any(grid[1, 1: -1]) or any(grid[-2, 1: -1]) or\
           any(grid[1: -1, 1]) or any(grid[1: -1, -2]):
            return
        yield i

To create a movie, we use the __FuncAnimation__ class from the __matplotlib.animation__ module. To create an object from that class, we need to provide the following arguments:
* The __matplotlib.pyplot.figure__ object to display in each frame, here __figure__ as previously defined.
* A function that takes one argument $v$. If $v$'s value is not __None__ then the function is expected to return a tuple of figure elements, usually modified, so that they can be appropriately redrawn for that frame; if $v$'s value is __None__ then no more frame is created and the movie is completed. Here that function is __evolve()__.

We make us of three extra keyword arguments:
* frames, whose default value is __itertools.count__, here set to __keep_going_or_stop__, called for each new frame and whose returned value is passed an argument to __evolve()__.
* __interval__, the time that elapses between the display of two successive frames, set by default to 200 (milliseconds), that we change to the value of __animation_intervals__.
* __save_count__, which because __keep_going_or_stop()__ is a generator function, is set by default to 100; we change that default to the value of __max_nb_of_iterations__.

In [ ]:
evolution = animation.FuncAnimation(figure, evolve,
                                    frames = keep_going_or_stop,
                                    interval = animation_intervals,
                                    save_count = max_nb_of_iterations
                                   )

Finally, we save __evolution__ as an __.mp4__ movie:

In [ ]:
evolution.save('gosperglidergun.mp4')

The program __b3_s23_movie.py__ has two additional features. First, it avoids overwriting an existing __.mp4__ file. Let us write a function that takes a file name as argument and creates a new file with the same name except possibly for its extension which we want to be __.mp4__. In case the file exists, the function adds before the __.mp4__ extension __\_1__, or __\_2__, or __\_3__... so that the resulting file does not exist. It is convenient to make use of the __sub()__ function from the __re__ module:

In [ ]:
re.sub('\..*', '', 'gosperglidergun.rle')
re.sub('\..*', '.mp4', 'gosperglidergun.rle')

To illustrate, we let the __system()__ function from the __os__ module execute the __touch__ command to create an empty file, whose name is provided as command line argument to __touch__. We also take advantage of the __ls__ and __rm__ Jupyter magic commands to list and remove, respectively, all files whose name starts with __my_rle_file__ in the working directory:

In [ ]:
def create_mp4_file(rle_filename):
    filename = re.sub('\..*', '', rle_filename)
    if os.path.isfile(filename + '.mp4'):
        for i in count(1):
            mp4_filename = ''.join((filename, '_', str(i), '.mp4'))
            if not os.path.isfile(mp4_filename):
                break
    else:
        mp4_filename = filename + '.mp4'
    os.system(f'touch {mp4_filename}')
        
os.system('touch my_rle_file.rle')
print('Files whose name starts with "my_rle_file":')
%ls my_rle_file*
for _ in range(3):
    create_mp4_file('my_rle_file.rle')
    print('\nFiles whose name starts with "my_rle_file":')
    %ls my_rle_file*
%rm my_rle_file*
print('\nFiles whose name starts with "my_rle_file":');

The additional feature of __b3_s23_movie.py__ is that it is meant to be run from the command line and accept command line arguments. For this purpose, we use the __ArgumentParser__ class from the __argparse__ module:

In [ ]:
parser = ArgumentParser()

parser

We use __ArgumentParser__'s  __add_argument()__'s method to add command line arguments. We provide as arguments to __add_argument()__ the name of the command line argument, which by convention is preceded by two hyphens if it is expected to itself take arguments. The __dest__ keyword argument is assigned the name of the Python variable that will store the value of that command line argument. The __required__ keyword argument has a default value of __False__. We change it to __True__ for only one command line argument:

In [ ]:
parser.add_argument('--rle_filename', dest = 'rle_filename', required = True)

All other command line arguments are optional and provide default values values. By default, command line arguments are of type __str__, so we use the __type__ keyword argument to change it to __int__. The __--figure_size__ command line arguments is meant to be followed by 2 arguments, which requires using the __nargs__ keyword argument to change the default of 1 to 2:

In [ ]:
parser.add_argument('--figure_size', dest = 'figure_size',
                    default = (10, 10), nargs = 2, type = int
                   )
parser.add_argument('--pattern_size_multiplier',
                    dest = 'pattern_size_multiplier',
                   )
parser.add_argument('--max_nb_of_iterations',
                    dest = 'max_nb_of_iterations',
                    default = 1_000, type = int
                   )
parser.add_argument('--animation_intervals',
                    dest = 'animation_intervals',
                    default = 20, type = int
                   )

Using the file __gosperglidergun.rle__, __b3_s23_movie.py__ can be executed from the command line
* with the minimal number of command line arguments, as: __python3 b3_s23_movie.py --rle_filename gosperglidergun.rle__
* with the maximal number of command line arguments, for instance as: __python3 b3_s23_movie.py --rle_filename gosperglidergun.rle --max_nb_of_iterations 100 --figure_size 2 2 --animation_intervals 50 --pattern_size_multiplier 2__